## SigNet Package test

**General idea** : test the functions defined in the module1.py file and make a first application.

**General plan** : 
1. Import the data using the DropBox dataset

2. Repeat the clustering several times to get several clustering outcomes

3. Compute the weights of each stocks within each cluster 

4. Compute the returns of each portfolio, where one portfolio corresponds to one clustering and is composed of 5 big assets (which are the 5 clusters) **in this notebook, we first fix the number of clusters to be equal to 5**

Here are the main package we use in this notebook

In [16]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from bs4 import BeautifulSoup
import requests 
from pypfopt.efficient_frontier import EfficientFrontier

In [17]:
pip install git+https://github.com/robertmartin8/PyPortfolioOpt.git

  Cloning https://github.com/robertmartin8/PyPortfolioOpt.git to /private/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/pip-req-build-gmh5w6q9
  Running command git clone --filter=blob:none --quiet https://github.com/robertmartin8/PyPortfolioOpt.git /private/var/folders/q2/wg5gyfhj2r9cd97zfmckktvw0000gn/T/pip-req-build-gmh5w6q9
  Resolved https://github.com/robertmartin8/PyPortfolioOpt.git to commit 4495242933fbbb7ffada242303a1e8c8700b974a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


We also import the module1.py file to use the function we defined there.

In [3]:
## we make some manipulations to correctly import module1

import os
import sys

# on a obtenu le chemin absolu vers notre répertoire en utilisant
# le terminal (ls, pwd, cd)
module_1_directory = '/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/code'

# Ajouter le chemin du répertoire 'code' au chemin de recherche Python
sys.path.append(module_1_directory)

# Maintenant, vous pouvez importer module1
import module1

### 1. Importation of data

We download various types of data (Open, High, Low, Close, Adj Close, Volume) for **496** assets between two periods (start, end) into a pandas dataframe denoted as df.

We then store the returns of each of these assets in the 'data'.

In [4]:
df = pd.read_csv('/Users/khelifanail/Documents/GitHub/Portfolio_clustering_project/Data/DATA_Statapp.csv')


In [9]:
df.head()

,ticker,open,high,low,close,volume,OPCL,pvCLCL,prevAdjClose,SPpvCLCL,sharesOut,PERMNO,SICCD,PERMCO,prevRawOpen,prevRawClose,prevAdjOpen
0,AA,"[82.0, 80.5, 82.0, 85.875, 86.0, 87.125, 82.0,...","[83.5625, 81.8125, 86.5, 86.375, 86.875, 87.25...","[80.375, 80.3125, 81.0, 84.8125, 84.5625, 84.3...","[80.9375, 81.3125, 86.0, 84.875, 84.625, 84.37...","[1551299, 2234799, 3121599, 4494699, 4534699, ...","[-0.013042, 0.010043, 0.047628, -0.011713, -0....","[-0.024849, 0.004633, 0.057648, -0.013081, -0....","[83.0, 80.94, 81.31, 86.0, 84.88, 84.62, 84.38...","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[366407, 366407, 366407, 366407, 366407, 36640...","[24643, 24643, 24643, 24643, 24643, 24643, 246...","[3334, 3334, '3334', 3334, 3334, '3334', '3334...","[20060, 20060, 20060, 20060, 20060, 20060, 200...","[80.8125, 82.0, 80.5, 82.0, 85.875, 86.0, 87.1...","[83.0, 80.9375, 81.3125, 86.0, 84.875, 84.625,...","[80.84, 82.01, 80.5, 82.09, 85.88, 86.01, 87.1..."
1,ABM,"[20.5, 20.125, 20.25, 20.1875, 20.1875, 20.25,...","[20.625, 20.375, 20.25, 20.375, 20.375, 20.25,...","[20.0, 20.0, 20.0, 20.0625, 20.0625, 20.0, 20....","[20.3125, 20.375, 20.125, 20.1875, 20.25, 20.2...","[120800, 62400, 27400, 63900, 60500, 113100, 3...","[-0.009188, 0.012346, -0.006192, 0.0, 0.003091...","[-0.003067, 0.003077, -0.01227, 0.003106, 0.00...","[20.37, 20.31, 20.38, 20.12, 20.19, 20.25, 20....","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[22341, 22341, 22341, 22341, 22341, 22341, 223...","[47730, 47730, 47730, 47730, 47730, 47730, 477...","[7349, 7349, '7349', 7349, 7349, '7349', '7349...","[20068, 20068, 20068, 20068, 20068, 20068, 200...","[20.1875, 20.5, 20.125, 20.25, 20.1875, 20.187...","[20.375, 20.3125, 20.375, 20.125, 20.1875, 20....","[20.19, 20.5, 20.13, 20.25, 20.19, 20.19, 20.2..."
2,ABT,"[35.25, 34.4375, 33.5625, 34.0, 34.5, 36.0, 34...","[36.0, 34.75, 34.3125, 35.25, 36.25, 36.0625, ...","[34.75, 33.75, 33.5625, 33.8125, 34.5, 34.875,...","[35.0, 34.0, 33.9375, 35.125, 35.5, 35.25, 34....","[4774099, 4818899, 5262299, 7846599, 7072899, ...","[-0.007117, -0.012786, 0.011111, 0.032553, 0.0...","[-0.036145, -0.028571, -0.001838, 0.034991, 0....","[36.31, 35.0, 34.0, 33.94, 35.13, 35.5, 35.25,...","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[1537311, 1537311, 1537311, 1537311, 1537311, ...","[20482, 20482, 20482, 20482, 20482, 20482, 204...","[2834, 2834, '2834', 2834, 2834, '2834', '2834...","[20017, 20017, 20017, 20017, 20017, 20017, 200...","[36.4375, 35.25, 34.4375, 33.5625, 34.0, 34.5,...","[36.3125, 35.0, 34.0, 33.9375, 35.125, 35.5, 3...","[36.44, 35.25, 34.44, 33.56, 34.02, 34.51, 36...."
3,ADI,"[93.5, 89.5, 85.625, 86.875, 84.0, 90.0, 93.5,...","[93.875, 91.5, 88.25, 87.625, 88.5, 94.75, 94....","[88.0, 85.5625, 83.1875, 83.25, 82.625, 89.25,...","[90.1875, 85.625, 86.875, 84.5, 86.875, 94.437...","[1827799, 1266599, 1614000, 1300500, 945300, 1...","[-0.036071, -0.044261, 0.014493, -0.027719, 0....","[-0.030242, -0.050589, 0.014599, -0.027338, 0....","[93.0, 90.19, 85.62, 86.87, 84.5, 86.88, 94.44...","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[174459, 174459, 174459, 174459, 174459, 17445...","[60871, 60871, 60871, 60871, 60871, 60871, 608...","[3612, 3612, '3612', 3612, 3612, '3612', '3612...","[282, 282, 282, 282, 282, 282, 282, 282, 282, ...","[91.5, 93.5, 89.5, 85.625, 86.875, 84.0, 90.0,...","[93.0, 90.1875, 85.625, 86.875, 84.5, 86.875, ...","[91.51, 93.56, 89.59, 85.63, 86.91, 84.05, 90...."
4,ADM,"[12.0, 11.8125, 11.875, 11.625, 11.875, 12.0, ...","[12.0625, 12.1875, 11.875, 11.875, 12.0, 12.18...","[11.875, 11.8125, 11.625, 11.5625, 11.8125, 11...","[12.0, 11.875, 11.6875, 11.75, 11.9375, 11.937...","[893200, 986900, 986800, 816300, 1076000, 1346...","[0.0, 0.005277, -0.015915, 0.010695, 0.005249,...","[-0.010309, -0.010417, -0.015789, 0.005348, 0....","[12.12, 12.0, 11.87, 11.69, 11.75, 11.94, 11.9...","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[608360, 608360, 608360, 608360, 60836

In [15]:
df.set_index(df.iloc[:,0])

open  \
open                                                                                                    
[82.0, 80.5, 82.0, 85.875, 86.0, 87.125, 82.0, ...  [82.0, 80.5, 82.0, 85.875, 86.0, 87.125, 82.0,...   
[20.5, 20.125, 20.25, 20.1875, 20.1875, 20.25, ...  [20.5, 20.125, 20.25, 20.1875, 20.1875, 20.25,...   
[35.25, 34.4375, 33.5625, 34.0, 34.5, 36.0, 34....  [35.25, 34.4375, 33.5625, 34.0, 34.5, 36.0, 34...   
[93.5, 89.5, 85.625, 86.875, 84.0, 90.0, 93.5, ...  [93.5, 89.5, 85.625, 86.875, 84.0, 90.0, 93.5,...   
[12.0, 11.8125, 11.875, 11.625, 11.875, 12.0, 1...  [12.0, 11.8125, 11.875, 11.625, 11.875, 12.0, ...   
...                                                                                               ...   
[31.0, 29.6875, 29.34375, 29.04688, 29.5, 30.53...  [31.0, 29.6875, 29.34375, 29.04688, 29.5, 30.5...   
[79.5, 77.375, 78.0, 80.625, 85.9375, 84.1875, ...  [79.5, 77.375, 78.0, 80.625, 85.9375, 84.1875,...   
[23.375, 24.1875, 23.0, 23.5625, 23.75, 24.75, ...  [23.375, 24.1875, 23.0, 23.5625, 23.75, 24.75,...   
[38.5625, 37.0, 36.75, 36.5, 36.4375, 35.5625, ...  [38.5625, 37.0, 36.75, 36.5, 36.4375, 35.5625,...   
[6.625, 6.5625, 6.5625, 6.625, 6.75, 6.75, 6.81...  [6.625, 6.5625, 6.5625, 6.625, 6.75, 6.75, 6.8...   

                                                                                                 high  \
open                                                                                                    
[82.0, 80.5, 82.0, 85.875, 86.0, 87.125, 82.0, ...  [83.5625, 81.8125, 86.5, 86.375, 86.875, 87.25...   
[20.5, 20.125, 20.25, 20.1875, 20.1875, 20.25, ...  [20.625, 20.375, 20.25, 20.375, 20.375, 20.25,...   
[35.25, 34.4375, 33.5625, 34.0, 34.5, 36.0, 34....  [36.0, 34.75, 34.3125, 35.25, 36.25, 36.0625, ...   
[93.5, 89.5, 85.625, 86.875, 84.0, 90.0, 93.5, ...  [93.875, 91.5, 88.25, 87.625, 88.5, 94.75, 94....   
[12.0, 11.8125, 11.875, 11.625, 11.875, 12.0, 1...  [12.0625, 12.1875, 11.875, 11.875, 12.0, 12.18...   
...                                                                                               ...   
[31.0, 29.6875, 29.34375, 29.04688, 29.5, 30.53...  [31.0, 29.8125, 29.34375, 29.40625, 30.5625, 3...   
[79.5, 77.375, 78.0, 80.625, 85.9375, 84.1875, ...  [80.75, 78.1875, 81.75, 85.8125, 86.25, 84.625...   
[23.375, 24.1875, 23.0, 23.5625, 23.75, 24.75, ...  [24.875, 24.1875, 23.5, 23.9375, 24.75, 24.75,...   
[38.5625, 37.0, 36.75, 36.5, 36.4375, 35.5625, ...  [38.5625, 37.5, 36.9375, 36.8125, 36.5, 37.5, ...   
[6.625, 6.5625, 6.5625, 6.625, 6.75, 6.75, 6.81...  [6.625, 6.6875, 6.6875, 6.8125, 6.8125, 6.9375...   

                                                                                                  low  \
open                                                                                                    
[82.0, 80.5, 82.0, 85.875, 86.0, 87.125, 82.0, ...  [80.375, 80.3125, 81.0, 84.8125, 84.5625, 84.3...   
[20.5, 20.125, 20.25, 20.1875, 20.1875, 20.25, ...  [20.0, 20.0, 20.0, 20.0625, 20.0625, 20.0, 20....   
[35.25, 34.4375, 33.5625, 34.0, 34.5, 36.0, 34....  [34.75, 33.75, 33.5625, 33.8125, 34.5, 34.875,...   
[93.5, 89.5, 85.625, 86.875, 84.0, 90.0, 93.5, ...  [88.0, 85.5625, 83.1875, 83.25, 82.625, 89.25,...   
[12.0, 11.8125, 11.875, 11.625, 11.875, 12.0, 1...  [11.875, 11.8125, 11.625, 11.5625, 11.8125, 11...   
...                                                                                               ...   
[31.0, 29.6875, 29.34375, 29.04688, 29.5, 30.53...  [29.85938, 29.03125, 28.71875, 29.0, 29.5, 30....   
[79.5, 77.375, 78.0, 80.625, 85.9375, 84.1875, ...  [77.875, 76.5, 77.8125, 80.1875, 84.625, 83.56...   
[23.375, 24.1875, 23.0, 23.5625, 23.75, 24.75, ...  [23.1875, 22.9375, 22.5625, 23.375, 23.0625, 2...   
[38.5625, 37.0, 36.75, 36.5, 36.4375, 35.5625, ...  [37.0625, 36.0, 36.5625, 36.375, 35.375, 35.5,...   
[6.625, 6.5625, 6.5625, 6.625, 6.75, 6.75, 6.81...  [6.5, 6.5625, 6.5625, 6.5625, 6.6875, 6.75, 6....   


In [14]:
df

,open,high,low,close,volume,OPCL,pvCLCL,prevAdjClose,SPpvCLCL,sharesOut,PERMNO,SICCD,PERMCO,prevRawOpen,prevRawClose,prevAdjOpen
0,"[82.0, 80.5, 82.0, 85.875, 86.0, 87.125, 82.0,...","[83.5625, 81.8125, 86.5, 86.375, 86.875, 87.25...","[80.375, 80.3125, 81.0, 84.8125, 84.5625, 84.3...","[80.9375, 81.3125, 86.0, 84.875, 84.625, 84.37...","[1551299, 2234799, 3121599, 4494699, 4534699, ...","[-0.013042, 0.010043, 0.047628, -0.011713, -0....","[-0.024849, 0.004633, 0.057648, -0.013081, -0....","[83.0, 80.94, 81.31, 86.0, 84.88, 84.62, 84.38...","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[366407, 366407, 366407, 366407, 366407, 36640...","[24643, 24643, 24643, 24643, 24643, 24643, 246...","[3334, 3334, '3334', 3334, 3334, '3334', '3334...","[20060, 20060, 20060, 20060, 20060, 20060, 200...","[80.8125, 82.0, 80.5, 82.0, 85.875, 86.0, 87.1...","[83.0, 80.9375, 81.3125, 86.0, 84.875, 84.625,...","[80.84, 82.01, 80.5, 82.09, 85.88, 86.01, 87.1..."
1,"[20.5, 20.125, 20.25, 20.1875, 20.1875, 20.25,...","[20.625, 20.375, 20.25, 20.375, 20.375, 20.25,...","[20.0, 20.0, 20.0, 20.0625, 20.0625, 20.0, 20....","[20.3125, 20.375, 20.125, 20.1875, 20.25, 20.2...","[120800, 62400, 27400, 63900, 60500, 113100, 3...","[-0.009188, 0.012346, -0.006192, 0.0, 0.003091...","[-0.003067, 0.003077, -0.01227, 0.003106, 0.00...","[20.37, 20.31, 20.38, 20.12, 20.19, 20.25, 20....","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[22341, 22341, 22341, 22341, 22341, 22341, 223...","[47730, 47730, 47730, 47730, 47730, 47730, 477...","[7349, 7349, '7349', 7349, 7349, '7349', '7349...","[20068, 20068, 20068, 20068, 20068, 20068, 200...","[20.1875, 20.5, 20.125, 20.25, 20.1875, 20.187...","[20.375, 20.3125, 20.375, 20.125, 20.1875, 20....","[20.19, 20.5, 20.13, 20.25, 20.19, 20.19, 20.2..."
2,"[35.25, 34.4375, 33.5625, 34.0, 34.5, 36.0, 34...","[36.0, 34.75, 34.3125, 35.25, 36.25, 36.0625, ...","[34.75, 33.75, 33.5625, 33.8125, 34.5, 34.875,...","[35.0, 34.0, 33.9375, 35.125, 35.5, 35.25, 34....","[4774099, 4818899, 5262299, 7846599, 7072899, ...","[-0.007117, -0.012786, 0.011111, 0.032553, 0.0...","[-0.036145, -0.028571, -0.001838, 0.034991, 0....","[36.31, 35.0, 34.0, 33.94, 35.13, 35.5, 35.25,...","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[1537311, 1537311, 1537311, 1537311, 1537311, ...","[20482, 20482, 20482, 20482, 20482, 20482, 204...","[2834, 2834, '2834', 2834, 2834, '2834', '2834...","[20017, 20017, 20017, 20017, 20017, 20017, 200...","[36.4375, 35.25, 34.4375, 33.5625, 34.0, 34.5,...","[36.3125, 35.0, 34.0, 33.9375, 35.125, 35.5, 3...","[36.44, 35.25, 34.44, 33.56, 34.02, 34.51, 36...."
3,"[93.5, 89.5, 85.625, 86.875, 84.0, 90.0, 93.5,...","[93.875, 91.5, 88.25, 87.625, 88.5, 94.75, 94....","[88.0, 85.5625, 83.1875, 83.25, 82.625, 89.25,...","[90.1875, 85.625, 86.875, 84.5, 86.875, 94.437...","[1827799, 1266599, 1614000, 1300500, 945300, 1...","[-0.036071, -0.044261, 0.014493, -0.027719, 0....","[-0.030242, -0.050589, 0.014599, -0.027338, 0....","[93.0, 90.19, 85.62, 86.87, 84.5, 86.88, 94.44...","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[174459, 174459, 174459, 174459, 174459, 17445...","[60871, 60871, 60871, 60871, 60871, 60871, 608...","[3612, 3612, '3612', 3612, 3612, '3612', '3612...","[282, 282, 282, 282, 282, 282, 282, 282, 282, ...","[91.5, 93.5, 89.5, 85.625, 86.875, 84.0, 90.0,...","[93.0, 90.1875, 85.625, 86.875, 84.5, 86.875, ...","[91.51, 93.56, 89.59, 85.63, 86.91, 84.05, 90...."
4,"[12.0, 11.8125, 11.875, 11.625, 11.875, 12.0, ...","[12.0625, 12.1875, 11.875, 11.875, 12.0, 12.18...","[11.875, 11.8125, 11.625, 11.5625, 11.8125, 11...","[12.0, 11.875, 11.6875, 11.75, 11.9375, 11.937...","[893200, 986900, 986800, 816300, 1076000, 1346...","[0.0, 0.005277, -0.015915, 0.010695, 0.005249,...","[-0.010309, -0.010417, -0.015789, 0.005348, 0....","[12.12, 12.0, 11.87, 11.69, 11.75, 11.94, 11.9...","[-0.009549, -0.038345, 0.001922, 0.000956, 0.0...","[608360, 608360, 608360, 608360, 608360, 60836...","[10516, 1051